In [2]:
import requests
import urllib.parse
from constants import HEADERS
import pandas as pd
from datetime import datetime

url = 'https://stats.nba.com/stats/drafthistory'
params = {
    'College': '',
    'LeagueID': '00',
    'OverallPick': '',
    'RoundNum': '',
    'RoundPick': '',
    'Season': '',
    'TeamID': '0',
    'TopX': '',
}
params_str = urllib.parse.urlencode(params, safe=":+")
response = requests.get(url=url, params=params_str, headers=HEADERS).json()
df = pd.json_normalize(response, 'resultSets')
df = pd.DataFrame(df["rowSet"][0], columns=df["headers"][0])

In [3]:
date_string = datetime.now().strftime("%Y%m%d")
df.to_csv(f'../data/alltime_draft_{date_string}.csv')

Player Season requesting

In [6]:
from player_season import PlayerSeasonRequester
import os
RequestObj = PlayerSeasonRequester()

In [7]:
start_year = 1989
end_year = 2022
years = range(start_year, end_year)
df_list = []
base_dir = 'D:/Andy/python/nba_analysis_app/data'
for year in years:
    season_end_year = year + 1
    season_id = f'{year}-{str(season_end_year)[-2:]}'
    RequestObj.populate_season(season_id=season_id)
    tmp_df = RequestObj.df
    tmp_df['season_id'] = season_id
    tmp_df['season_year_start'] = year
    df_list.append(RequestObj.df)
    outfile = '/'.join([base_dir, 'player_season_stats', f'{season_id}.csv'])
    tmp_df.to_csv(outfile)
    print('completed {season_id} season'.format(season_id=season_id))

completed 1989-90 season
completed 1990-91 season
completed 1991-92 season
completed 1992-93 season
completed 1993-94 season
completed 1994-95 season
completed 1995-96 season
completed 1996-97 season
completed 1997-98 season
completed 1998-99 season
completed 1999-00 season
completed 2000-01 season
completed 2001-02 season
completed 2002-03 season
completed 2003-04 season
completed 2004-05 season
completed 2005-06 season
completed 2006-07 season
completed 2007-08 season
completed 2008-09 season
completed 2009-10 season
completed 2010-11 season
completed 2011-12 season
completed 2012-13 season
completed 2013-14 season
completed 2014-15 season
completed 2015-16 season
completed 2016-17 season
completed 2017-18 season
completed 2018-19 season
completed 2019-20 season
completed 2020-21 season
completed 2021-22 season


In [8]:
import pandas as pd
df_tot = pd.concat(df_list)

Here we need to add a column that indicates what year it is for the player (e.g. 1, 2, 3, etc...)

In [15]:
draft_df = pd.read_csv('D://Andy/python/nba_analysis_app/data/alltime_draft.csv')
draft_df.rename(columns={'PERSON_ID': 'PLAYER_ID'}, inplace=True)

In [42]:
# joine dataframes and calculate player year
combo_df = pd.merge(df_tot, draft_df, on=['PLAYER_ID'], how='left')
combo_df['player_year'] = (combo_df.season_year_start - combo_df.SEASON) + 1
logic = (combo_df.player_year == 1) & (combo_df.TEAM_ID_x != combo_df.TEAM_ID_y)
combo_df.loc[logic, 'TEAM_ID_x'] = combo_df.loc[logic, 'TEAM_ID_y']
combo_df.loc[logic, 'TEAM_ABBREVIATION_x'] = combo_df.loc[logic, 'TEAM_ABBREVIATION_y']

In [44]:
combo_df.to_csv('D:/Andy/python/nba_analysis_app/data/1989-21_combo.csv')